In [1]:
import numpy as np  
from sklearn.manifold import TSNE  
from sklearn.model_selection import train_test_split  
from sklearn.datasets import fetch_openml  
from sklearn.preprocessing import StandardScaler  
from tensorflow.keras.models import Sequential  
from tensorflow.keras.layers import Dense, Concatenate  
from tensorflow.keras.utils import to_categorical

In [2]:
# 加载数据集  
mnist = fetch_openml('mnist_784', version=1)  
X, y = mnist.data, mnist.target  
X = X / 255.0  # 归一化数据  
y = np.array([label for label in y])  
y = to_categorical(y)  # 将标签转换为one-hot编码 

D:\Users\Administrator\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\datasets\_openml.py:1002: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [4]:
# 划分数据集  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
# 对原始数据进行标准化  
scaler = StandardScaler()  
X_train_scaled = scaler.fit_transform(X_train)  
X_test_scaled = scaler.transform(X_test) 

In [7]:
X_all_scaled = np.vstack((X_train_scaled, X_test_scaled)) 

In [8]:
# 使用t-SNE进行降维  
tsne = TSNE(n_components=2, random_state=0)  
# 对整个数据集进行降维  
X_all_tsne = tsne.fit_transform(X_all_scaled) 

In [9]:
# 根据原始数据的索引分割回训练集和测试集 
n_train = X_train_scaled.shape[0]

X_train_tsne = X_all_tsne[:n_train, :]  
X_test_tsne = X_all_tsne[n_train:, :] 

In [10]:
# 将原始特征和降维后的特征堆叠起来  
X_train_combined = np.concatenate((X_train_scaled, X_train_tsne), axis=1)  
X_test_combined = np.concatenate((X_test_scaled, X_test_tsne), axis=1)

In [15]:
from keras.layers import Input, Dense, Concatenate  
from keras.models import Model

In [46]:
# 定义原始特征和t-SNE特征的输入层  
input_original = Input(shape=(784,), name='original_input')
input_tsne = Input(shape=(2,), name='tsne_input')
  
# 定义隐藏层，对每个输入进行处理  
dense_original = Dense(64, activation='relu')(input_original)  
dense_tsne = Dense(64, activation='relu')(input_tsne)  
  
# 连接两个隐藏层的输出  
combined = Concatenate()([dense_original, dense_tsne])  
  
# 定义输出层  
output = Dense(10)(combined)  
  
# 创建模型，输入是原始特征和t-SNE特征，输出是分类结果  
model = Model(inputs=[input_original, input_tsne], outputs=output)

In [47]:
# 编译模型  
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])  

In [37]:
y_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

In [49]:
# 训练模型  
X_train_original = X_train_combined[:, :784]  
X_train_tsne = X_train_combined[:, 784:]  
X_test_original = X_test_combined[:, :784]  
X_test_tsne = X_test_combined[:, 784:]  
model.fit([X_train_original, X_train_tsne], y_train, epochs=10, batch_size=32,validation_data=([X_test_original,X_test_tsne], y_test))

Epoch 1/10
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.3122 - loss: 9.5691 - val_accuracy: 0.1951 - val_loss: 8.0644
Epoch 2/10
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.2440 - loss: 10.1877 - val_accuracy: 0.3491 - val_loss: 10.8616
Epoch 3/10
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.3049 - loss: nan - val_accuracy: 0.0959 - val_loss: nan
Epoch 4/10
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.0965 - loss: nan - val_accuracy: 0.0959 - val_loss: nan
Epoch 5/10
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.1007 - loss: nan - val_accuracy: 0.0959 - val_loss: nan
Epoch 6/10
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.0981 - loss: nan - val_accuracy: 0.0959 - val_loss: nan
Epoch 7/10
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.0996 - loss: nan - val_accuracy: 0.0959 - val_loss: nan
Epoch 8/10
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.1008 - loss: nan - val_accuracy: 0.0959 - val_loss

In [51]:
# 评估模型  
_, accuracy = model.evaluate([X_test_original,X_test_tsne], y_test)  
print(f'Test accuracy: {accuracy}')

438/438 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0980 - loss: nan
Test accuracy: 0.09592857211828232
